In [1]:
from nltk.corpus import stopwords
import nltk
from nltk import word_tokenize, FreqDist
from nltk.stem import WordNetLemmatizer
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
from gensim.models import Word2Vec
import pandas as pd

In [2]:
# nltk.download('words')

In [3]:
# !pip install gensim

In [4]:
df = pd.read_csv('base_elon_data.csv')

* Need to clean tweets

In [5]:
df.head()

,Time,retweet_from,tweet_length,encoded_sentiment,sentiment,polarity,tweet
0,2017-09-29 17:39:19,NaN,105,1,positive,0.366667,"Assuming max acceleration of 2 to 3 g's, but i..."
1,2017-09-29 10:44:54,SpaceX,113,1,positive,0.200000,BFR is capable of transporting satellites to o...
2,2017-09-29 10:39:57,NaN,6,0,neutral,0.000000,Yup :)
3,2017-09-29 09:56:12,NaN,7,0,neutral,0.000000,Part 2
4,2017-09-29 09:19:21,NaN,96,1,positive,0.650000,Fly to most places on Earth in under 30 mins a...


In [6]:
dfZ = df.dropna(subset=['tweet','tweet_length'])

In [7]:
dfZ.iloc[2]

Time                 2017-09-29 10:39:57
retweet_from                         NaN
tweet_length                           6
encoded_sentiment                      0
sentiment                        neutral
polarity                               0
tweet                             Yup :)
Name: 2, dtype: object

In [8]:
# len(df[df['tweet_length']==0])

In [9]:
dfZ.isna().sum()

Time                    0
retweet_from         2682
tweet_length            0
encoded_sentiment       0
sentiment               0
polarity                0
tweet                   0
dtype: int64

In [10]:
# for row in dfZ['Time']:
#     row == row[0:10]

In [11]:
# dfZ.Time

In [12]:
dfZ.describe()

,tweet_length,encoded_sentiment,polarity
count,3205.000000,3205.000000,3205.000000
mean,74.749142,0.344462,0.124443
std,40.438685,0.702171,0.263477
min,1.000000,-1.000000,-1.000000
25%,40.000000,0.000000,0.000000
50%,80.000000,0.000000,0.000000
75%,110.000000,1.000000,0.250000
max,140.000000,1.000000,1.000000


In [13]:
tweets = dfZ['tweet'].tolist()

In [14]:
type(tweets)

list

In [15]:
wordnet_lemmatizer = WordNetLemmatizer()
english = set(nltk.corpus.words.words()+ ['yup'])

In [16]:
# english'yup'}

In [17]:
# def lemmadata2(doc):
#         pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
#         raw_tokens = nltk.regexp_tokenize(doc, pattern)
#         tokens = [i.lower() for i in raw_tokens]
#         lemmatized = [wordnet_lemmatizer.lemmatize(word, pos="v") for word in tokens]
#         words = list(filter(lambda w: w in english, lemmatized))
#         return words

### Dropping Filter to keep "slang" words

In [18]:
def lemmadata(doc):
        pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
        raw_tokens = nltk.regexp_tokenize(doc, pattern)
        tokens = [i.lower() for i in raw_tokens]
        lemmatized = [wordnet_lemmatizer.lemmatize(word) for word in tokens]
        words = list(filter(lambda w: w in english, lemmatized))
        return words

In [19]:
def lemmadata(doc):
    pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
    raw_tokens = nltk.regexp_tokenize(doc, pattern)
    tokens = [i.lower() for i in raw_tokens]
    stop_words = set(stopwords.words('english'))
    listed = [w for w in tokens if not w in stop_words]
    lemmatized = [wordnet_lemmatizer.lemmatize(word) for word in listed]
#     lemmatized = list(filter(lambda w: w != 'lb', lemmatized))
    words = list(filter(lambda w: w in english, lemmatized))
    return words

In [20]:
lemmadata(tweets[2])

['yup']

In [21]:
tweets[:10]

["Assuming max acceleration of 2 to 3 g's, but in a comfortable direction. Will feel like a mild to moder? ",
 'BFR is capable of transporting satellites to orbit, crew and cargo to the and completing missions to the Moon an?',
 'Yup :)',
 'Part 2 ',
 'Fly to most places on Earth in under 30 mins and anywhere in under 60. Cost per seat should be? ',
 'Supporting the creation of a permanent, self-sustaining human presence on Mars. ',
 'BFR will take you anywhere on Earth in less than 60 mins ',
 'Mars City Opposite of Earth. Dawn and dusk sky are blue on Mars and day sky is red. ',
 'Moon Base Alpha ',
 "Will be announcing something really special at today's talk "]

In [22]:
lemmatized = [lemmadata(post) for post in tweets]

In [23]:
lemmadata(tweets[0])

['assuming',
 'acceleration',
 'comfortable',
 'direction',
 'feel',
 'like',
 'mild']

In [24]:
processed_data = list(map(lemmadata, dfZ['tweet']))

In [25]:
processed_data

[['assuming',
  'acceleration',
  'comfortable',
  'direction',
  'feel',
  'like',
  'mild'],
 ['capable',
  'transporting',
  'satellite',
  'orbit',
  'crew',
  'cargo',
  'mission',
  'moon'],
 ['yup'],
 ['part'],
 ['fly', 'place', 'earth', 'min', 'anywhere', 'cost', 'per', 'seat'],
 ['supporting',
  'creation',
  'permanent',
  'self',
  'sustaining',
  'human',
  'presence',
  'mar'],
 ['take', 'anywhere', 'earth', 'min'],
 ['mar',
  'city',
  'opposite',
  'earth',
  'dawn',
  'dusk',
  'sky',
  'blue',
  'mar',
  'day',
  'sky',
  'red'],
 ['moon', 'base', 'alpha'],
 ['something', 'really', 'special', 'talk'],
 ['nine',
  'year',
  'ago',
  'today',
  'falcon',
  'first',
  'privately',
  'liquid',
  'fuel',
  'rocket',
  'orbit',
  'earth'],
 ['another', 'day', 'office'],
 ['love'],
 ['excited', 'say', 'officially', 'beauty', 'age'],
 ['idea',
  'paying',
  'b',
  'dev',
  'cost',
  'underpants',
  'turned',
  'um',
  'lucrative'],
 ['headed',
  'soon',
  'describe',
  'new',


In [26]:
# newlist = [item for items in processed_data for item in items]

In [27]:
# newlist

## processed data = list of lists of lemmatized tokens 
### lemmatization part of speech processing = verb (might change to noun)

In [28]:
processed_data[0:2]

[['assuming',
  'acceleration',
  'comfortable',
  'direction',
  'feel',
  'like',
  'mild'],
 ['capable',
  'transporting',
  'satellite',
  'orbit',
  'crew',
  'cargo',
  'mission',
  'moon']]

In [29]:
# lemmatized[0]

In [30]:
model = Word2Vec(processed_data, min_count = 1)

In [31]:
model.wv.most_similar('rocket')

[('landing', 0.9580685496330261),
 ('good', 0.9559077620506287),
 ('car', 0.9547291398048401),
 ('like', 0.9530527591705322),
 ('model', 0.9515939354896545),
 ('launch', 0.9513512849807739),
 ('w', 0.9512720108032227),
 ('falcon', 0.9509298205375671),
 ('lot', 0.9491704702377319),
 ('need', 0.9491258859634399)]

In [32]:
X = model[model.wv.vocab]

/Users/flatironschool/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [33]:
X

array([[-4.4168727e-03, -1.7240405e-03,  3.4652741e-04, ...,
        -7.6088990e-04,  1.3821470e-03, -5.7414351e-03],
       [-4.4340831e-03,  5.8970796e-03,  1.1731152e-03, ...,
         1.0559770e-03,  1.0728098e-05, -2.1790119e-03],
       [ 1.8099144e-03, -8.1011391e-04, -3.2061585e-03, ...,
         3.1114183e-03,  1.1297405e-03, -3.5689790e-03],
       ...,
       [-3.7258348e-04,  3.4298914e-04, -4.5015579e-04, ...,
         1.5809634e-03, -4.3255105e-03,  2.1094999e-03],
       [ 4.5940103e-03, -3.6323920e-03,  3.8481259e-03, ...,
         3.1692628e-03, -1.3246784e-03,  1.3715660e-04],
       [ 4.2636776e-03, -3.0103771e-03,  2.1528876e-03, ...,
        -4.1564708e-03, -1.8942947e-03,  4.1027586e-03]], dtype=float32)

In [34]:
model.wv.index2word

['model',
 'car',
 'launch',
 'good',
 'rocket',
 'like',
 'falcon',
 'yes',
 'first',
 'next',
 'landing',
 'year',
 'x',
 'would',
 'time',
 'new',
 'one',
 'thanks',
 'u',
 'dragon',
 'great',
 'week',
 'v',
 'need',
 'w',
 'autopilot',
 'make',
 'go',
 'right',
 'look',
 'test',
 'much',
 'best',
 'people',
 'also',
 'solar',
 'mar',
 'high',
 'get',
 'stage',
 'yeah',
 'flight',
 'day',
 'really',
 'soon',
 'way',
 'today',
 'maybe',
 'love',
 'work',
 'lot',
 'battery',
 'coming',
 'ai',
 'system',
 'article',
 'ever',
 'mile',
 'team',
 'space',
 'company',
 'still',
 'video',
 'back',
 'mission',
 'long',
 'better',
 'actually',
 'sure',
 'month',
 'future',
 'live',
 'change',
 'exactly',
 'going',
 'production',
 'probably',
 'true',
 'want',
 'even',
 'thing',
 'think',
 'orbit',
 'say',
 'almost',
 'satellite',
 'owner',
 'k',
 'drive',
 'point',
 'many',
 'b',
 'supercharger',
 'energy',
 'cool',
 'p',
 'power',
 'last',
 'know',
 'full',
 'take',
 'world',
 'end',
 'well'

In [35]:
# summarize vocabulary
words = list(model.wv.vocab)

In [36]:
words

['assuming',
 'acceleration',
 'comfortable',
 'direction',
 'feel',
 'like',
 'mild',
 'capable',
 'transporting',
 'satellite',
 'orbit',
 'crew',
 'cargo',
 'mission',
 'moon',
 'yup',
 'part',
 'fly',
 'place',
 'earth',
 'min',
 'anywhere',
 'cost',
 'per',
 'seat',
 'supporting',
 'creation',
 'permanent',
 'self',
 'sustaining',
 'human',
 'presence',
 'mar',
 'take',
 'city',
 'opposite',
 'dawn',
 'dusk',
 'sky',
 'blue',
 'day',
 'red',
 'base',
 'alpha',
 'something',
 'really',
 'special',
 'talk',
 'nine',
 'year',
 'ago',
 'today',
 'falcon',
 'first',
 'privately',
 'liquid',
 'fuel',
 'rocket',
 'another',
 'office',
 'love',
 'excited',
 'say',
 'officially',
 'beauty',
 'age',
 'idea',
 'paying',
 'b',
 'dev',
 'underpants',
 'turned',
 'um',
 'lucrative',
 'headed',
 'soon',
 'describe',
 'new',
 'planetary',
 'colonizer',
 'design',
 'detail',
 'worth',
 'seeing',
 'right',
 'yes',
 'good',
 'article',
 'several',
 'value',
 'fiction',
 'simulation',
 'interplanetar

In [37]:
print(model['china'])

[-2.5910048e-03  6.6132932e-03 -1.5343912e-03 -3.1291542e-03
 -5.2294746e-04  4.1645751e-03  3.9023985e-03 -2.2288710e-03
  2.0105436e-03  3.3348061e-03  2.3610581e-04  4.2075431e-03
 -5.0518563e-04 -1.9405712e-03 -5.5942277e-04  6.7899106e-03
 -4.5764497e-03 -4.0496471e-03 -4.7987946e-03 -3.1078616e-03
 -6.4447857e-03  3.6128184e-03 -3.6297606e-03 -1.4125646e-04
 -3.5690116e-03 -1.7179846e-03  3.6201088e-03 -2.5610637e-04
  7.8151532e-04  2.1403881e-03  4.9142991e-03  1.9827133e-04
 -6.7096334e-03  3.1660025e-03  3.3399954e-03 -5.9849448e-03
 -3.0788241e-03  1.0798504e-03  1.6917205e-04  2.1955878e-03
  2.6695128e-03 -2.5536644e-03  4.9311873e-03  2.9424299e-04
  1.5085300e-04  2.5184180e-03 -3.8341186e-03 -2.9071507e-03
  8.5905027e-03 -2.0227057e-03  9.7909058e-04 -3.8112858e-03
 -1.6245986e-03  7.8248495e-04 -1.5966772e-03 -2.4548357e-03
 -1.2747837e-04 -8.1370228e-05  6.0714723e-04  2.2444459e-05
  1.0520190e-03  4.2358236e-03 -2.1906951e-04  1.3079140e-03
 -2.4313656e-04 -1.84988

/Users/flatironschool/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [38]:
import pandas as pd

In [39]:
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
import numpy as np
import gensim

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

In [40]:
model

In [41]:
max_size = len(model.wv.vocab)-1

In [42]:
w2v = np.zeros((max_size,model.layer1_size))

/Users/flatironschool/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `layer1_size` (Attribute will be removed in 4.0.0, use self.trainables.layer1_size instead).
  """Entry point for launching an IPython kernel.


In [43]:
w2v

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [44]:
with open("metadata.tsv", 'w+') as file_metadata:
    for i,word in enumerate(model.wv.index2word[:max_size]):
        w2v[i] = model.wv[word]
        file_metadata.write(word + '\n')

In [45]:
w2v

array([[-0.00086225,  0.01607456, -0.02196649, ..., -0.01004225,
        -0.00242024, -0.01977327],
       [-0.00245489,  0.0177294 , -0.01724802, ..., -0.01411749,
        -0.00451943, -0.02233617],
       [-0.00261409,  0.01675385, -0.01959996, ..., -0.00483227,
        -0.00160694, -0.02377649],
       ...,
       [-0.0048069 ,  0.0002341 ,  0.00276477, ...,  0.00406864,
        -0.00308153, -0.00342892],
       [-0.00037258,  0.00034299, -0.00045016, ...,  0.00158096,
        -0.00432551,  0.0021095 ],
       [ 0.00459401, -0.00363239,  0.00384813, ...,  0.00316926,
        -0.00132468,  0.00013716]])

In [46]:
sess = tf.InteractiveSession()

In [47]:
#Let us create a 2D tensor called embedding that holds our embeddings.
with tf.device("/cpu:0"):
    embedding = tf.Variable(w2v, trainable=False, name='embedding')

In [48]:
tf.global_variables_initializer().run()

In [49]:
path = 'tensorboard'

In [50]:
# let us create an object to Saver class which is actually used to 
#save and restore variables to and from our checkpoints
saver = tf.train.Saver()

In [51]:
# using file writer, we can save our summaries and events to our event file.
writer = tf.summary.FileWriter(path, sess.graph)

In [52]:
# adding into projector
config = projector.ProjectorConfig()
embed = config.embeddings.add()
embed.tensor_name = 'embedding'
embed.metadata_path = 'metadata.tsv'

In [53]:
# Specify the width and height of a single thumbnail.
projector.visualize_embeddings(writer, config)

saver.save(sess, path+'/model.ckpt', global_step=max_size)

'tensorboard/model.ckpt-3762'